# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [42]:
# import libraries
import pandas as pd
import nltk
import re

from sqlalchemy import create_engine
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lindstaedt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lindstaedt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lindstaedt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)
X = df.message
Y = df.drop(['message', 'original'], axis=1)

In [49]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [50]:
Y.head()

,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,direct,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,direct,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [51]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # build multi-language list of stop words
    stop_words = [*stopwords.words("english"),
                  *stopwords.words("french"),
                  *stopwords.words("spanish")]
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

In [52]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [63]:
# initialize tf-idf vectorizer object
vectorizer = TfidfVectorizer(tokenizer=tokenize)

In [64]:
# compute bag of word counts and tf-idf values
XTFIDF = vectorizer.fit_transform(X.head())

In [65]:
vectorizer.vocabulary_

{'weather': 29,
 'update': 28,
 'cold': 2,
 'front': 9,
 'cuba': 6,
 'could': 3,
 'pas': 18,
 'haiti': 11,
 'hurricane': 13,
 'looking': 15,
 'someone': 23,
 'name': 16,
 'report': 19,
 'leogane': 14,
 '80': 0,
 '90': 1,
 'destroyed': 8,
 'hospital': 12,
 'st': 24,
 'croix': 5,
 'functioning': 10,
 'need': 17,
 'supply': 25,
 'desperately': 7,
 'say': 21,
 'west': 30,
 'side': 22,
 'rest': 20,
 'country': 4,
 'today': 26,
 'tonight': 27}

In [66]:
# convert sparse matrix to numpy array to view
XTFIDF.toarray()

array([[0.        , 0.        , 0.36152912, 0.36152912, 0.        ,
        0.        , 0.36152912, 0.        , 0.        , 0.36152912,
        0.        , 0.29167942, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.36152912, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.36152912, 0.36152912,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = make_pipeline(
                MultiOutputClassifier(AdaBoostClassifier)
                        )

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.